In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


## STANFORD METHOD

Import the files:

In [ ]:
CGM_FILE = 'CGM.csv'
INSULIN_FILE = 'Insulin.csv'

def load_data(cgm_file, insulin_file):
    """
    Load CGM and insulin data from CSV files.
    """
    cgm_data = pd.read_csv(cgm_file, parse_dates=['Time'])
    insulin_data = pd.read_csv(insulin_file, parse_dates=['Time'])
    
    return cgm_data, insulin_data

load_data(CGM_FILE, INSULIN_FILE)

First identify correction boluses

In [ ]:
CGM_filtered_250=cgm_data[cgm_data['glucose']>250]
CGM_filtered_300=cgm_data[cgm_data['glucose']>300]

if 'timestamp' not in CGM_filtered_250.columns:
    CGM_filtered_250 = CGM_filtered_250.reset_index()

if 'timestamp' not in CGM_filtered_300.columns:
    CGM_filtered_300 = CGM_filtered_300.reset_index()

CGM_filtered_250 = CGM_filtered_250.copy()
CGM_filtered_300 = CGM_filtered_300.copy()


CGM_filtered_250['timestamp'] = pd.to_datetime(CGM_filtered_250['timestamp'])
CGM_filtered_300['timestamp'] = pd.to_datetime(CGM_filtered_300['timestamp'])

if CGM_filtered_250.index.name != 'timestamp':
    CGM_filtered_250 = CGM_filtered_250.set_index('timestamp')

if CGM_filtered_300.index.name != 'timestamp':
    CGM_filtered_300 = CGM_filtered_300.set_index('timestamp')

def bolus_between_CGM(cgm_data, bolus_timestamp):
    cgm_before = cgm_data[cgm_data.index < bolus_timestamp].tail(1)
    cgm_after = cgm_data[cgm_data.index > bolus_timestamp].head(1)
    
    if not cgm_before.empty and not cgm_after.empty:
        return cgm_before, cgm_after
    else:
        return None, None

insulin_data_cleaned = insulin_data.dropna(subset=['bolus_insulin']).copy()
insulin_data_cleaned['timestamp'] = pd.to_datetime(insulin_data_cleaned['timestamp'])
insulin_data_cleaned = insulin_data_cleaned.drop(columns=['infused','minutes','Unnamed: 0','delayed_insulin', 'duration'])
insulin_data_cleaned.set_index('timestamp', inplace=True)
correction_bolus_1 = pd.DataFrame()
correction_bolus_2 = pd.DataFrame()
cgm_before_300_acc = []
cgm_before_250_acc = []

if not insulin_data_cleaned.empty:
    for idx, bolus in insulin_data_cleaned.iterrows():
        bolus_timestamp = bolus.name  
        cgm_before, cgm_after = bolus_between_CGM(cgm_data, bolus_timestamp)
        if cgm_before is not None and not cgm_before.empty and cgm_after is not None and not cgm_after.empty:
            if cgm_before['glucose'].iloc[-1] >300 and cgm_after['glucose'].iloc[-1] >300:
                correction_bolus_1 = pd.concat([correction_bolus_1, pd.DataFrame([bolus])], ignore_index=False)
                cgm_before_300_acc.append(cgm_before['glucose'].iloc[-1])
        if cgm_before is not None and not cgm_before.empty and cgm_after is not None and not cgm_after.empty:          
            if cgm_before['glucose'].iloc[-1] >250 and cgm_after['glucose'].iloc[-1] >250:
                correction_bolus_2 = pd.concat([correction_bolus_2, pd.DataFrame([bolus])], ignore_index=False)
                cgm_before_250_acc.append(cgm_before['glucose'].iloc[-1])
            
correction_bolus_1['cgm_before_300']=cgm_before_300_acc
correction_bolus_2['cgm_before_250']=cgm_before_250_acc
print('IDENTIFIED CORRECTION BOLUSES (CGM>300)')
print('---------------------------------------')
print(correction_bolus_1)
print('')
print('IDENTIFIED CORRECTION BOLUSES (CGM>250)')
print('---------------------------------------')
print(correction_bolus_2)

    

IDENTIFIED CORRECTION BOLUSES (CGM>300)
---------------------------------------
                     insulin  bolus_insulin  insulin_from_delayed         day  \
2022-01-28 22:20:24    400.0          400.0                   0.0  2022-01-28   
2022-02-07 16:02:29    450.0          450.0                   0.0  2022-02-07   
2022-02-10 22:53:53    400.0          400.0                   0.0  2022-02-10   
2022-02-13 10:51:48    450.0          450.0                   0.0  2022-02-13   
2022-02-13 19:54:58    600.0          600.0                   0.0  2022-02-13   

                     cgm_before_300  
2022-01-28 22:20:24           319.0  
2022-02-07 16:02:29           373.0  
2022-02-10 22:53:53           330.0  
2022-02-13 10:51:48           341.0  
2022-02-13 19:54:58           321.0  

IDENTIFIED CORRECTION BOLUSES (CGM>250)
---------------------------------------
                     insulin  bolus_insulin  insulin_from_delayed         day  \
2022-01-16 14:27:53    200.0          200.0

Not decreasing 50mg/dL an hour after a correction bolus for a blood glucose level greater than 300mg/dL.
Some patients over 250mg/dL, try both.

In [ ]:
stanford_failures_1 = []
stanford_failures_2 = []

for idx, correction in correction_bolus_1.iterrows():
    hour_away = idx + timedelta(hours=1)
    cgm_value = cgm_data.loc[cgm_data.index >= hour_away, 'glucose']
    if not cgm_value.empty:
        if cgm_value.iloc[0] < correction['cgm_before_300'] - 50:
            stanford_failures_1.append(idx)

for idx, correction in correction_bolus_2.iterrows():
    hour_away = idx + timedelta(hours=1)
    cgm_value = cgm_data.loc[cgm_data.index >= hour_away, 'glucose']
    if not cgm_value.empty:
        if cgm_value.iloc[0] < correction['cgm_before_250'] - 50:
            stanford_failures_2.append(idx)
days_1 = [ts.date() for ts in stanford_failures_1]
days_2 = [ts.date() for ts in stanford_failures_2]

times_1 = [ts for ts in stanford_failures_1]
times_2 = [ts for ts in stanford_failures_2]

# Timestamps for the failures
print('STANFORD FAILURES (CGM>300)')
print('---------------------------------------')
print(times_1)
print('STANFORD FAILURES (CGM>250)')
print('---------------------------------------')
print(times_2)

[datetime.date(2022, 2, 7), datetime.date(2022, 2, 13)]
[datetime.date(2022, 1, 16), datetime.date(2022, 1, 24), datetime.date(2022, 1, 27), datetime.date(2022, 2, 7), datetime.date(2022, 2, 13), datetime.date(2022, 2, 14)]
[Timestamp('2022-02-07 16:02:29'), Timestamp('2022-02-13 10:51:48')]
[Timestamp('2022-01-16 14:27:53'), Timestamp('2022-01-24 16:21:31'), Timestamp('2022-01-27 10:51:10'), Timestamp('2022-02-07 16:02:29'), Timestamp('2022-02-13 10:51:48'), Timestamp('2022-02-14 07:51:50')]
